In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import codecs
import os
import pandas as pd
import numpy as np
import regex
from collections import Counter
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import re 

In [34]:
bible_ver_dic = {}

new_bib = pd.read_csv("./data/bible/t_web.csv", index_col = [0])
old_bib = pd.read_csv("./data/bible/t_kjv.csv", index_col = [0])
literal_bib = pd.read_csv("./data/bible/t_ylt.csv", index_col = [0])

new_bib.t = new_bib.t.str.lower()
old_bib.t = old_bib.t.str.lower()
literal_bib.t = literal_bib.t.str.lower()


# a regex to remove the curly braces and the notes inside them. 
def remove_notes(notes):
    return regex.sub("[\{\[].*?[\}\]]", "", notes)

# remove notes from the bible text
new_bib.t = new_bib.t.apply(remove_notes)
old_bib.t = old_bib.t.apply(remove_notes)
literal_bib.t = literal_bib.t.apply(remove_notes)

bibles = new_bib.copy()
bibles['new_bib'] = new_bib.t
bibles['old_bib'] = old_bib.t
bibles['literal_bib'] = literal_bib.t
bibles = bibles.drop(columns='t')

np.where(bibles.applymap(lambda x: x == ''))[0]


empty_rows = bibles[bibles['new_bib'] == ''].index
bibles = bibles.drop(empty_rows)
np.where(bibles.applymap(lambda x: x == ''))[0]



bibles = bibles.sample(frac=1)
print(bibles.shape)
bibles.head()
bibles.to_csv('./data/preprocessed/bibles_suffled.csv')

train = round(31096 * 0.7)
val = round(31096 * 0.2)
test = round(31096 * 0.1)
print(train, val, test, 'sum:',train+test+val)

(31096, 6)
21767 6219 3110 sum: 31096


In [38]:


new_bib.t = new_bib.t.apply(text_cleaner)



In [39]:
new_bib

,b,c,v,t
id,,,,
1001001,1,1,1,in the beginning god created the heavens and t...
1001002,1,1,2,now the earth was formless and empty darkness ...
1001003,1,1,3,god said let there be light and there was light
1001004,1,1,4,god saw the light and saw that it was good god...
1001005,1,1,5,god called the light day and the darkness he c...
...,...,...,...,...
66022017,66,22,17,the spirit and the bride say come he who hears...
66022018,66,22,18,testify to everyone who hears the words of the...
66022019,66,22,19,if anyone takes away from the words of the boo...


In [25]:
train = round(31096 * 0.7)
val = round(31096 * 0.2)
test = round(31096 * 0.1)
print(train, val, test, 'sum:',train+test+val)

21767 6219 3110 sum: 31096


In [26]:
train_df = bibles[0:train].copy()
val_df = bibles[train:val+train].copy()
test_df = bibles[val+train:test+val+train].copy()


In [27]:
train_df['trg'] = train_df['literal_bib']
train_df['src'] = train_df['new_bib']
train_df = train_df.drop(columns = ['b', 'c', 'v', 'new_bib', 'old_bib', 'literal_bib'])

val_df['trg'] = val_df['literal_bib']
val_df['src'] = val_df['new_bib']
val_df = val_df.drop(columns = ['b', 'c', 'v', 'new_bib', 'old_bib', 'literal_bib'])

test_df['trg'] = test_df['literal_bib']
test_df['src'] = test_df['new_bib']
test_df = test_df.drop(columns = ['b', 'c', 'v', 'new_bib', 'old_bib', 'literal_bib'])

In [30]:
train_df.to_csv('./data/preprocessed/train_df.csv')
val_df.to_csv('./data/preprocessed/val_df.csv')
test_df.to_csv('./data/preprocessed/test_df.csv')